In [ ]:
from datamodule.datamodule import *
from distributions.distributions import *
from utils import *
# MODEL:
from model.deepGARv1 import DeepGAR
# PYTORCH LIGHTNING:
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
#

In [2]:

name = 'electric'  # electric
train_size = 0.7
validation_size = 0.15
test_size = 0.15
batch_size = 64
custom_datamodule = DataModule(name, (train_size, validation_size, test_size), batch_size=batch_size)

ehi


LD2011_2014.txt.zip: 261MB [00:14, 17.9MB/s]                              


In [3]:
train_window = 24
test_window = 24
test_horizon = 24
test_stride = 24 # should be equal to the time horizon
test_delay = 0
train_loader, val_loader, test_loader = custom_datamodule.get_all(
    window=train_window,
    test_window=test_window,
    test_horizon=test_horizon,
    test_stride=test_stride,
    test_delay=test_delay
)

In [ ]:
input_size = custom_datamodule.get_channels()
n_nodes = custom_datamodule.get_number_of_nodes()

distribution = GaussianDistribution()  # or StudentTDistribution()

test_loss = 'rmse'  # or 'mae'

perform_scaling = False

In [ ]:
def run_model(run, train=False):

    model = DeepGAR(
        input_size=input_size,
        n_nodes=n_nodes,
        distribution=distribution,
        test_loss=test_loss,
        perform_scaling=perform_scaling
    )
    print(model)
    print_model_size(model)

    early_stopping_callback = EarlyStopping(
        monitor='val_loss',
        min_delta=0.00,
        patience=20,
        mode='min'
    )
    checkpoint_callback = ModelCheckpoint(
        dirpath=f'electricity_logs_{run}',
        save_top_k=1,
        monitor='val_loss',
        mode='min',
    )
    trainer = pl.Trainer(
        max_epochs=400,
        accelerator='cuda' if torch.cuda.is_available() else 'cpu',
        devices = 1,
        callbacks=[checkpoint_callback, early_stopping_callback]
    )
    if trainer:
        trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)
    # delete all
    try:
        del model
        del early_stopping_callback
        del checkpoint_callback
        del trainer
    except:
        pass


# list_available_models()

In [ ]:
train_n_times=1
for i in range(train_n_times):
    run_model(i, train=False)

In [ ]:
# TESTING

In [ ]:
best_model_path = checkpoint_callback.best_model_path
print(best_model_path)

best_model = DeepGAR.load_from_checkpoint(
    checkpoint_path=best_model_path
)
best_model.eval()
res = trainer.predict(model=best_model, dataloaders=test_loader)